In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
# plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float', "{:.4f}".format)

In [11]:
train = pd.read_csv('data/TrainAndValid.csv', low_memory=False)
test = pd.read_csv('data/test.csv', low_memory=False)

In [12]:
train['saledate'] = pd.to_datetime(train.saledate)
test['saledate'] = pd.to_datetime(test.saledate)

In [13]:
train.set_index('saledate', inplace=True)
train.sort_index(inplace=True)
train.index.name = None

test.set_index('saledate', inplace=True)
test.sort_index(inplace=True)
test.index.name = None